In [1]:
# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import time

In [2]:
# Chargement et prétraitement des données
final_merged_df = pd.read_csv('data/processed/data_init.csv', index_col=0)

X = final_merged_df.drop(columns=['grav'])
y = final_merged_df['grav']

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Création d'un ensemble d'entraînement et d'un ensemble test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=12)

# Définition du classificateur
LR = LogisticRegression(random_state=123, tol=1e-3, solver='saga')

# Validation croisée
cv3 = KFold(n_splits=3, random_state=111, shuffle=True)

# Définition de l'espace de recherche des hyperparamètres
param_grid = [
    {'C': [0.1, 1, 10], 'penalty': ['l1'], 'max_iter': [1000]},
    {'C': [0.1, 1, 10], 'penalty': ['l2'], 'max_iter': [1000]},
    {'C': [0.1, 1, 10], 'penalty': ['elasticnet'], 'max_iter': [1000], 'l1_ratio': np.linspace(0, 1, 6)}
]

# Création d'un GridSearchCV personnalisé pour sauvegarder les résultats partiels
class GridSearchWithProgress(GridSearchCV):
    def fit(self, X, y=None, **fit_params):
        n_candidates = sum(len(ParameterGrid(grid)) for grid in self.param_grid)
        n_completed = 0
        
        results = []
        
        for params in ParameterGrid(self.param_grid):
            start_time = time.time()
            self.estimator.set_params(**params)
            cv_results = cross_validate(self.estimator, X, y, cv=self.cv, scoring=self.scoring, return_train_score=True)
            end_time = time.time()
            
            result = {
                'params': params,
                'mean_test_score': cv_results['test_score'].mean()
            }
            
            results.append(result)
            
            n_completed += 1
            
            # Enregistrer les résultats partiels
            joblib.dump(results, 'gridcv_results_LR.joblib')
            
            # Affichage des résultats partiels
            print(f"Résultat partiel {n_completed}/{n_candidates}:")
            print(f"Paramètres: {params}")
            print(f"Score moyen: {cv_results['test_score'].mean():.4f} (+/- {cv_results['test_score'].std():.4f})")
            print(f"Temps de fit: {end_time - start_time:.2f} secondes")
            print()
            
        self.cv_results_ = results
        best_index = max(range(len(results)), key=lambda i: results[i]['mean_test_score'])
        self.best_params_ = results[best_index]['params']
        self.best_score_ = results[best_index]['mean_test_score']
        
        # Définition du meilleur estimateur
        self.best_estimator_ = self.estimator.set_params(**self.best_params_)
        self.best_estimator_.fit(X, y)
        
        return self

# Exécution du GridSearchCV avec contrôle de l'état d'avancement
grid_search = GridSearchWithProgress(LR, param_grid, cv=cv3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Chargement des résultats enregistrés
results = joblib.load('gridcv_results_LR.joblib')

Résultat partiel 1/24:
Paramètres: {'C': 0.1, 'max_iter': 1000, 'penalty': 'l1'}
Score moyen: 0.5894 (+/- 0.0008)
Temps de fit: 10495.11 secondes

Résultat partiel 2/24:
Paramètres: {'C': 1, 'max_iter': 1000, 'penalty': 'l1'}
Score moyen: 0.5895 (+/- 0.0008)
Temps de fit: 13500.07 secondes

Résultat partiel 3/24:
Paramètres: {'C': 10, 'max_iter': 1000, 'penalty': 'l1'}
Score moyen: 0.5895 (+/- 0.0008)
Temps de fit: 14126.99 secondes

Résultat partiel 4/24:
Paramètres: {'C': 0.1, 'max_iter': 1000, 'penalty': 'l2'}
Score moyen: 0.5895 (+/- 0.0008)
Temps de fit: 7520.46 secondes

Résultat partiel 5/24:
Paramètres: {'C': 1, 'max_iter': 1000, 'penalty': 'l2'}
Score moyen: 0.5895 (+/- 0.0008)
Temps de fit: 7513.58 secondes

Résultat partiel 6/24:
Paramètres: {'C': 10, 'max_iter': 1000, 'penalty': 'l2'}
Score moyen: 0.5895 (+/- 0.0008)
Temps de fit: 7510.78 secondes

Résultat partiel 7/24:
Paramètres: {'C': 0.1, 'l1_ratio': 0.0, 'max_iter': 1000, 'penalty': 'elasticnet'}
Score moyen: 0.5895 (

In [3]:
# Utilisation du meilleur modèle
best_model = grid_search.best_estimator_

In [4]:
# Sauvegarde du meilleur modèle
joblib.dump(best_model, 'best_model_LR.joblib')

['best_model_LR.joblib']

In [5]:
# Affichage des meilleurs paramètres et du meilleur score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score:", grid_search.best_score_)

# Utilisation du meilleur modèle
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print("Accuracy du meilleur modèle:", accuracy_score(y_test, y_pred_best))

Meilleurs paramètres: {'C': 1, 'l1_ratio': 0.4, 'max_iter': 1000, 'penalty': 'elasticnet'}
Meilleur score: 0.5895009263286214
Accuracy du meilleur modèle: 0.5923140628339388


In [6]:
# Rapport de classification détaillé pour le meilleur modèle
print("\nRapport de classification du meilleur modèle:\n", classification_report(y_test, y_pred_best, zero_division=0))


Rapport de classification du meilleur modèle:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        29
           1       0.62      0.83      0.71     31181
           2       0.00      0.00      0.00      2078
           3       0.51      0.07      0.12     11552
           4       0.56      0.59      0.58     30024

    accuracy                           0.59     74864
   macro avg       0.34      0.30      0.28     74864
weighted avg       0.56      0.59      0.55     74864

